#### Data Science Project - Machine Leaning.
Predicating Stroke Population based on 5 different factors.

Factors:Stroke as the target.
Diabetes, Heart Disease, High Blood Pressure, High Cholesterol, and Obesity.

####### Data provided by https://data.cdc.gov/
Dataset Files:  Diabetes.cvs
Data Measure: 500 Cities: Diagnosed diabetes among adults aged >=18 years

Capstone: 
Y = (Predicted) Stroke. 
X = (Multiple Factors) Diabetes, Heart Dissease, High Blood Pressure, High Cholesterol, and Obesity 

ML Algorithms = Will be focusing on supervised (multiple linear regression) numeric values.

Data type/Metrix:
Quantitative/Nemerical Data (Continuous)
Discrete data - Data with final set of vlaues which can be categorized.
Continuous - take any numerical value within a range. 
-=============================================
Intructions:
-=============================================
1.	filter by, StateCity and Age-adjusted prevalence
2.	create three data frames based on distinct GeographicLevel
3.	Join or copy the US row to materialize the ratios for US
4.	Join the city data by cityFIPS to the tractFIPS rows to calculate their ratios
5.	Split the geo into (lat , long)
6.	join all features into one dataframe
7.	join one filtered frame of stroke target before 
8.	modeling


-=========================================
Data Wranging: 
**** Instruction #1 and #5
-=========================================
1. Drop all unwanted columns.
   Keeping and sort in the following orders: 
   Short_Question_Text,	StateAbbr,	CityName,	Data_Value,	Low_Confidence_Limit,	High_Confidence_Limit,	GeoLocation,


2. Sort data (Asce order)
 - by StateAbbr, StateDesc, CityName,
 - Data_Value_Type(order by Age-adjusted prevalence) assigned to  DB_New_df.
3. DB_New_df merge 3column names and create a new column called Db_StCity.
  Db_StCity = Short_Question_Text + StateAbbr + CityName.  
4. Split the GeoLocation into:
    Latitude = New column [DB_Latitude ]
                42.536457 
    Longitude = New column [DB_Longitude ]
   	            -70.985786
5. Format: missing values or removing unwanted records(dup), rearrange columns, change data type...ect.
  ** Remove null/blank in the column with str. before merging rows.
  ** Check for inconsist names in the rows. 
  ** For example:  the state name is TX but the cell name is "United States".
  *** Check for Geolocation, it "Blank" sometimes.

6. Write to a new file, " NEW_Diabetes.csv"
7. Repeat the same process for all five files.
   -======================================
8. Run stat and Data Calculations:
9. Merge data frames.



  






In [ ]:

# importing the libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import math 
#%metplotlib inline #


In [ ]:
%pip install scikit-learn  --upgrade --force
#from sklearn.linear_model import LinearRegression
#from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
# Reading, loading  datasets. 

df = pd.read_csv("500_Cities_Diagnosed_diabetes.csv" )


In [ ]:
df.shape 

In [ ]:
df.head().T


In [ ]:
df.columns

In [26]:
# Dropping unwanted columns.
New_df = df.drop(columns=['Year','StateDesc', 'DataSource', 'Category', 'UniqueID', \
    'Data_Value_Unit','DataValueTypeID',  'Measure', 'Data_Value_Footnote_Symbol', \
    'Data_Value_Footnote', 'CategoryID', 'MeasureId', 'Short_Question_Text'])

#Rename columns
New_df = New_df.rename(columns={'StateAbbr':'Db_States', 'CityName':'Db_City',\ 
    'GeographicLevel':'Db_Geoloc', 'Data_Value_Type':'Db_AgeAdj_preval', \
    'Data_Value':'Db_Mean_Pop', 'Low_Confidence_Limit':'Db_Low_Conf', \
    'High_Confidence_Limit':'Db_High_Conf', 'PopulationCount':'Db_Population' })

#Change data type:
# removing the "," from the PopCountByMM values and replace it with nothing than change it from object type to int. 
New_df["Db_Population"] = New_df["Db_Population"].str.replace(',','').astype(int)


In [27]:
New_df.columns

Index(['Db_States', 'Db_City', 'Db_Geoloc', 'Db_AgeAdj_preval', 'Db_Mean_Pop',
       'Db_Low_Conf', 'Db_High_Conf', 'Db_Population', 'GeoLocation',
       'CityFIPS', 'TractFIPS'],
      dtype='object')

In [ ]:
New_df.columns

In [ ]:
#New_df.sort_values(["States"])
New_df.sort_values(["Db_States", "City"], ascending=[1,0])

In [ ]:
# Checking for NaN/NULL and count them.
#New_df.isna().sum()
# Checking for NULL, NaN values
#New_df.query("Pre_Diabetic.isna()")

#Dropping NaN values. Make sure to select every columns in the dataframe.
New_df[[ 'States', 'City', 'AgeAdj_Prevalence', 'Diabetic', 'Low_Risk', 'Pre_Diabetic',
'Pop_Count' ]].dropna()


In [ ]:
# Double check on the NaN
New_df.query("Diabetic.isna()")
New_df

In [ ]:
#Double check on the NaN NULL
New_df.isna().sum()


In [ ]:
#New_df.sort_values(["States"])
New_df.sort_values(["States", "City"], ascending=[1,0])

In [ ]:
# write to a new file.
df_diab_01 = New_df.query('`AgeAdj_Prevalence` == "Age-adjusted prevalence"')
df_diab_01

# This code will do the same
#df_diab_01.loc[df_diab_01.duplicated(subset=['States'])]


In [ ]:
# Count dup values.
# df_diab_01.loc[df_diab_01.duplicated(subset=['States'])]
df_diab_01.loc[df_diab_01.duplicated(subset=['States'])].count()

In [ ]:
# Verify vales for TX/city based on "Age-adjusted prevalence"
df_diab_01 = df_diab_01.query('`States` == "TX"')
df_diab_01

In [ ]:
# Concat State and City name into a NEW column called StateAndCity:
df_diab_01 ['StateAndCity'] = df_diab_01['States'] + '_' + df_diab_01['City']
df_diab_01

In [ ]:
# DROPPING the states and city columns after concat:
df_diab_01 = df_diab_01.drop(columns=['States','City' ])
df_diab_01


In [ ]:
df_diab_01.columns

In [ ]:
#Reorganize the columns
#Create a new list in the order that you want.
New_cols = ['StateAndCity','Diabetic', 'Pre_Diabetic', 'Low_Risk', 'Pop_Count']
df_diab_01 = df_diab_01[New_cols]
df_diab_01

df_diab_01.to_csv("New_Diabetes_X01.csv")

In [ ]:
df_diab_01.describe().T